# Quick C3D data check for Colab

**Ebook:** A Hands-On Guide to Biomechanics Data Analysis with Python and AI  
**Author:** Dr. Hossein Mokhtarzadeh  
**Powered by:** PoseIQ™

This notebook loads sample biomechanics data and shows how to bring it into Python.
Click Runtime -> Run all.


In [ ]:
# Install dependencies
!pip install -q ezc3d pandas


In [ ]:
# Imports and sample downloader
import os, urllib.request, zipfile
import numpy as np
import matplotlib.pyplot as plt
import ezc3d

# Optional sample download. Works on Colab without hardcoded /content/... guesses.
def download_sample():
    base_dir = "/content/sample_data/c3d_zip"
    os.makedirs(base_dir, exist_ok=True)
    url = "https://www.c3d.org/data/Sample01.zip"
    zip_path = os.path.join(base_dir, "Sample01.zip")
    urllib.request.urlretrieve(url, zip_path)
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(base_dir)
    # Pick the first .c3d found
    c3d_candidates = []
    for root, _, files in os.walk(base_dir):
        for f in files:
            if f.lower().endswith(".c3d"):
                c3d_candidates.append(os.path.join(root, f))
    if not c3d_candidates:
        raise FileNotFoundError("No .c3d files found in the downloaded archive.")
    print("Extracted to:", base_dir)
    print("Found file:", os.path.basename(c3d_candidates[0]))
    return c3d_candidates[0]

# Choose your file:
# 1) Use the sample
path = download_sample()
# 2) Or set your own file path instead of the line above:
# path = "/content/your_file.c3d"


In [ ]:
def quick_check_c3d(path, show_plots=True):
    print("="*60)
    print("Quick C3D check")
    print("File:", path)

    try:
        c3d = ezc3d.c3d(path)
    except Exception as e:
        print("Could not open file. Error:", str(e))
        return

    # Metadata
    try:
        n_points = int(c3d['parameters']['POINT']['USED']['value'][0])
    except Exception:
        n_points = len(c3d['data']['points'][0]) if 'points' in c3d['data'] else 0

    try:
        # analogs is a list of subframes: [array(nAnalogs, nFrames), ...]
        if len(c3d['data']['analogs']) > 0:
            n_analogs = c3d['data']['analogs'][0].shape[0]
        else:
            n_analogs = 0
    except Exception:
        n_analogs = 0

    try:
        point_rate = float(c3d['parameters']['POINT']['RATE']['value'][0])
    except Exception:
        point_rate = float('nan')

    print("\nMetadata")
    print("Markers count:", n_points)
    print("Analogs count:", n_analogs)
    print("Point frame rate:", point_rate)

    if n_points == 0:
        print("Note: No markers found. The file may not be usable for kinematics.")
    if n_analogs == 0:
        print("Note: No analog channels found. Force or EMG may be missing.")
    if not np.isfinite(point_rate) or point_rate <= 0:
        print("Note: Point frame rate missing or invalid.")

    # Marker check on first marker
    print("\nMarkers quick check")
    points = c3d['data']['points'] if 'points' in c3d['data'] else None  # (4, nMarkers, nFrames)
    if points is not None and n_points > 0 and points.shape[2] > 0:
        x = points[0,0,:]
        y = points[1,0,:]
        z = points[2,0,:]
        w = points[3,0,:]  # often residual or a scale term

        finite_mask = np.isfinite(x) & np.isfinite(y) & np.isfinite(z)
        finite_ratio = float(np.mean(finite_mask))
        print("First marker finite ratio:", round(finite_ratio, 3))

        if finite_ratio < 0.9:
            print("Note: Many missing or invalid samples in the first marker.")

        # Simple continuity check
        x_clean = x[np.isfinite(x)]
        if x_clean.size > 1:
            dx = np.abs(np.diff(x_clean))
            med = np.nanmedian(dx) if dx.size else 0.0
            jump_ratio = float(np.mean(dx > (med * 20))) if med > 0 else 0.0
        else:
            jump_ratio = 0.0

        if jump_ratio > 0.05:
            print("Note: Marker shows sudden jumps. Consider gap fill or smoothing.")

        # Range sanity
        rng = np.nanmax([
            np.nanmax(x) - np.nanmin(x),
            np.nanmax(y) - np.nanmin(y),
            np.nanmax(z) - np.nanmin(z),
        ])
        if np.isfinite(rng) and rng == 0:
            print("Note: Marker appears flat. May be constant or not moving.")
        else:
            print("First marker range looks nonzero.")
    else:
        print("No marker data to check.")

    # Analogs quick check on first channel
    print("\nAnalogs quick check")
    try:
        analogs_list = c3d['data']['analogs']
    except Exception:
        analogs_list = []
    if n_analogs > 0 and len(analogs_list) > 0:
        analogs = analogs_list[0]   # shape (nAnalogs, nFrames)
        a0 = analogs[0, :] if analogs.shape[0] > 0 else np.array([])
        if a0.size:
            finite_ratio_a = float(np.mean(np.isfinite(a0)))
            print("First analog finite ratio:", round(finite_ratio_a, 3))
            if finite_ratio_a < 0.95:
                print("Note: Many missing or invalid samples in the first analog channel.")

            a0_min = float(np.nanmin(a0))
            a0_max = float(np.nanmax(a0))
            a0_std = float(np.nanstd(a0))
            print("First analog stats  min:", round(a0_min, 3),
                  " max:", round(a0_max, 3),
                  " std:", round(a0_std, 3))

            if a0_std == 0:
                print("Note: Analog channel is flat. Check device or scaling.")
            if not np.isfinite(a0_min) or not np.isfinite(a0_max):
                print("Note: Analog range invalid. Possible NaN values.")
        else:
            print("No analog samples available.")
    else:
        print("No analog data to check.")

    # Plots: first marker XY and first analog channel
    if show_plots:
        try:
            if points is not None and n_points > 0 and points.shape[2] > 0:
                plt.figure()
                plt.plot(points[0,0,:], points[1,0,:])
                plt.title("First marker XY trajectory quick view")
                plt.xlabel("X")
                plt.ylabel("Y")
                plt.show()
        except Exception as e:
            print("Could not plot marker. Error:", str(e))

        try:
            if n_analogs > 0 and len(analogs_list) > 0 and analogs_list[0].shape[0] > 0:
                plt.figure()
                plt.plot(analogs_list[0][0, :])
                plt.title("First analog channel quick view")
                plt.xlabel("Sample")
                plt.ylabel("Value")
                plt.show()
        except Exception as e:
            print("Could not plot analog. Error:", str(e))

    print("\nPlain English summary")
    print("The file opened and metadata printed. You saw how many markers and analog channels you have and the frame rate.")
    print("The first marker plot should look like a smooth path. A flat line or random jumps suggests missing points or tracking issues.")
    print("The first analog plot should show a clear signal like force pulses during contact. A flat line or extreme spikes suggests scaling or device issues.")
    print("This is a quick visual check only. For deeper confidence use a visual tool such as Mokka or Vicon Nexus and apply proper preprocessing before detailed analysis.")
    print("="*60)

# Example call
quick_check_c3d(path)  # e.g. quick_check_c3d('/content/sample_data/c3d_zip/Eb015pi.c3d')
